In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [10]:
# Loading trained model, scalar pickle, encoders pickle
model=load_model('model.h5')

with open('label_encoder_geography.pkl','rb') as file:
    label_encoder_geography=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open ('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)


In [13]:
# Sample input data

input_data = {
    'CreditScore': 600,
    'Geography': 'Germany',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [14]:
# Encoding Geography similar to previous preprocessing before training the model
geography_encoded = label_encoder_geography.transform([[input_data['Geography']]]).toarray()
df_geography_encoded = pd.DataFrame(geography_encoded, columns = label_encoder_geography.get_feature_names_out(['Geography']))
df_geography_encoded

/Users/aditya/Library/CloudStorage/OneDrive-Personal/AIR OneDrive/GenAI/ANN Project - Churn Classification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [18]:
df_input=pd.DataFrame([input_data])
df_input

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Germany,Male,40,3,60000,2,1,1,50000


In [19]:
# Encoding Gender column
df_input['Gender'] = label_encoder_gender.transform([[input_data['Gender']]])
df_input


/Users/aditya/Library/CloudStorage/OneDrive-Personal/AIR OneDrive/GenAI/ANN Project - Churn Classification/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Germany,1,40,3,60000,2,1,1,50000


In [22]:
df_input = df_input.drop(['Geography'], axis=1)
df_input

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1,40,3,60000,2,1,1,50000


In [25]:
#concat
df_input=pd.concat([df_input, df_geography_encoded], axis=1)
df_input

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,0.0,1.0,0.0


In [27]:
# Scaling the input
scaled_input = scaler.transform(df_input)
scaled_input

array([[-0.51359305,  0.89906522,  0.11503871, -0.71111292, -0.23486192,
         0.8027857 ,  0.65543311,  0.99302433, -0.87415516, -0.9990005 ,
         1.69365639, -0.56503205]])

In [28]:
# Predicting the churn
prediction = model.predict(scaled_input)
prediction

1/1 [==============================] - 0s 80ms/step


array([[0.04159089]], dtype=float32)

In [29]:
prediction_probability = prediction[0][0]
prediction_probability

0.04159089

In [30]:
if (prediction_probability > 0.5):
    print('Customer is likely to churn')
else:
    print('Customer is not likely to churn')

Customer is not likely to churn
